<a href="https://colab.research.google.com/github/CynicDog/delta-lake-lab/blob/main/Delta_Lake_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y pyspark delta-spark dataproc-spark-connect
!pip install -q pyspark==3.5.1 delta-spark==3.2.0

def get_spark():
    from pyspark.sql import SparkSession
    from delta import configure_spark_with_delta_pip

    builder = (
        SparkSession.builder.appName("DeltaLakeApp")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    )

    return configure_spark_with_delta_pip(builder).getOrCreate()

spark = get_spark()
spark

Found existing installation: pyspark 3.5.1
Uninstalling pyspark-3.5.1:
  Successfully uninstalled pyspark-3.5.1
Found existing installation: dataproc-spark-connect 0.8.3
Uninstalling dataproc-spark-connect-0.8.3:
  Successfully uninstalled dataproc-spark-connect-0.8.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

# Sample DataFrame
data = [
    (1, "Alice", 100),
    (2, "Bob", 200),
    (3, "Charlie", 300)
]

columns = ["id", "name", "amount"]

df = spark.createDataFrame(data, columns)
df.show()

delta_path = "/tmp/delta-table"

df.write.format("delta").mode("overwrite").save(delta_path)
print(f"Delta table saved at {delta_path}")

+---+-------+------+
| id|   name|amount|
+---+-------+------+
|  1|  Alice|   100|
|  2|    Bob|   200|
|  3|Charlie|   300|
+---+-------+------+

Delta table saved at /tmp/delta-table


In [4]:
delta_df = spark.read.format("delta").load(delta_path)
delta_df.show()

+---+-------+------+
| id|   name|amount|
+---+-------+------+
|  2|    Bob|   200|
|  3|Charlie|   300|
|  1|  Alice|   100|
+---+-------+------+

